In [1]:
# Text classification Example

# Data Load

import csv

file = open ("./IMDB_Dataset.csv", "r", encoding='UTF-8')  # ./ 기본경로는 Script가 저장되는 장소
rdr = csv.reader(file)

sentence = []
label = []

for index, line in enumerate(rdr):
    if index == 0:
        continue
    sentence.append(line[0].strip()) # sentence 리스트에 리뷰 내용 추가, strip() 함수는 문자열 양쪽의 공백 제거 (x)
    label.append(line[1].strip())    # label 리스트에 긍정, 부정 추가 (y)
    
# print data, not preprocessing (원본)
print("원본 검사 생략")
#print(sentence[0])

원본 검사 생략


In [2]:
# stemming 라이브러리 선언 및 테스트

from __future__ import print_function
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
#print(stemmer.stem("hook")) # stemmer test

In [3]:
# re 라이브러리로 특수문자 제거
import re

# 저장용 임시 리스트
#temp_data = [len(sentence)]
t_data=[]
# 소문자로 변환 .lower(), 불필요한 특수문자 제거
for i in range(len(sentence)):
    sentence[i] = sentence[i].lower() # lower() : 모든 문자를 소문자로 변환
    sentence[i] = re.sub("(<br\s*/><br\s*/>)|(\-)|(\/)", ' ', sentence[i])
    sentence[i] = re.sub("[.;:!\'?,\"()|[\]]", ' ', sentence[i])

In [4]:
# 단어 단위로 분할
result=[]
for i in sentence:
    result.append(i.split()) # 워드 단위로 분할 후 리스트에 저장
    
print(result[0])

['one', 'of', 'the', 'other', 'reviewers', 'has', 'mentioned', 'that', 'after', 'watching', 'just', '1', 'oz', 'episode', 'you', 'll', 'be', 'hooked', 'they', 'are', 'right', 'as', 'this', 'is', 'exactly', 'what', 'happened', 'with', 'me', 'the', 'first', 'thing', 'that', 'struck', 'me', 'about', 'oz', 'was', 'its', 'brutality', 'and', 'unflinching', 'scenes', 'of', 'violence', 'which', 'set', 'in', 'right', 'from', 'the', 'word', 'go', 'trust', 'me', 'this', 'is', 'not', 'a', 'show', 'for', 'the', 'faint', 'hearted', 'or', 'timid', 'this', 'show', 'pulls', 'no', 'punches', 'with', 'regards', 'to', 'drugs', 'sex', 'or', 'violence', 'its', 'is', 'hardcore', 'in', 'the', 'classic', 'use', 'of', 'the', 'word', 'it', 'is', 'called', 'oz', 'as', 'that', 'is', 'the', 'nickname', 'given', 'to', 'the', 'oswald', 'maximum', 'security', 'state', 'penitentary', 'it', 'focuses', 'mainly', 'on', 'emerald', 'city', 'an', 'experimental', 'section', 'of', 'the', 'prison', 'where', 'all', 'the', 'cells

In [5]:
sentence=[] # 결과를 받기위해 초기화
for i in range(len(result)):
    tmp=[]
    string=""
    for j in result[i]:
        tmp.append(stemmer.stem(j)) # stemming
    
    for k in tmp:
        string= string+k+' ' # 문자열로 추합
        
    sentence.append(string.rstrip()) # 좌우 불필요 공백 제거후 추합
print(sentence[0])

one of the other review has mention that after watch just 1 oz episod you ll be hook they are right as this is exact what happen with me the first thing that struck me about oz was it brutal and unflinch scene of violenc which set in right from the word go trust me this is not a show for the faint heart or timid this show pull no punch with regard to drug sex or violenc it is hardcor in the classic use of the word it is call oz as that is the nicknam given to the oswald maximum secur state penitentari it focus main on emerald citi an experiment section of the prison where all the cell have glass front and face inward so privaci is not high on the agenda em citi is home to mani aryan muslim gangsta latino christian italian irish and more so scuffl death stare dodgi deal and shadi agreement are never far away i would say the main appeal of the show is due to the fact that it goe where other show wouldn t dare forget pretti pictur paint for mainstream audienc forget charm forget romanc oz

In [6]:
# split data

# 학습 데이터에 40000 할당 (총 50000)
reviews_train_clean = sentence[:40000]
train_y = label[:40000]

# 나머지 10000은 검증 데이터 할당
reviews_test_clean = sentence[40000:]
test_y = label[40000:]

In [7]:
# CountVectorizer : 문서 집합에서 단어 토큰을 생성, 각 단어의 수를 세어 bag of words로 인코딩한 벡터 생성(단어 빈도 수 측정용도?)

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True) # 성능 개선을 위한 binary 이진분류 옵션
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)



In [8]:
# LogistiRegression 회귀분석을 이용한 최적의 파라미터 찾기


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, train_y, test_size = 0.25, random_state = 17)

for c in[0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C = c)
    lr.fit(X_train, y_train)
    print("C = ", c,", Accuracy :", accuracy_score(y_val, lr.predict(X_val)))

c:\python\venv\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


C =  0.01 , Accuracy : 0.8754
C =  0.05 , Accuracy : 0.8817
C =  0.25 , Accuracy : 0.8794
C =  0.5 , Accuracy : 0.875
C =  1 , Accuracy : 0.8736


In [9]:
final_model = LogisticRegression(C=0.05)
final_model.fit(X_train, y_train)

print("Final Accuracy : ", accuracy_score(test_y, final_model.predict(X_test)))

Final Accuracy :  0.8795


In [10]:
reviews_new = ['This movie was excellent', 'absolute joy ride',
              'Steven Seagal was terrible', 'Steven Seagal shined through.',
              'this was certainly a movie', 'Two thumbs up', 'I fell asleep halfway through',
              "We can't wait for the sequel!!", 'I can recommend this highly enough',
              'instant classic.', 'Steven Seagal was amazing. His perfomance was Oscar-worthy.',
              'I love it']

reviews_new_counts = cv.transform(reviews_new)

In [11]:
pred = final_model.predict(reviews_new_counts)

for review, category in zip(reviews_new,pred):
    print(review," -> ", category)

This movie was excellent  ->  negative
absolute joy ride  ->  positive
Steven Seagal was terrible  ->  negative
Steven Seagal shined through.  ->  negative
this was certainly a movie  ->  negative
Two thumbs up  ->  positive
I fell asleep halfway through  ->  negative
We can't wait for the sequel!!  ->  positive
I can recommend this highly enough  ->  positive
instant classic.  ->  positive
Steven Seagal was amazing. His perfomance was Oscar-worthy.  ->  negative
I love it  ->  positive
